In [1]:
import sys
import logging
from datetime import datetime, timedelta

import pandas as pd

from ai_data_generator import AiDataGenerator

from settings import Setting

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
	format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

In [2]:
setting = Setting()

In [3]:
ai_data_generator = AiDataGenerator(setting, logging)
current_time = datetime.now() - timedelta(days =2)

In [4]:
# ai_data_generator.generate(current_time, current_time)

In [5]:
data_df = pd.read_csv('traning_data.csv')

In [6]:
for i in range(1, len(data_df)):
    if data_df.loc[i, 'pe_oi_change'] == 1 and data_df.loc[i-1, 'pe_oi_change'] not in [0, 1]:
        data_df.loc[i, 'pe_oi_change'] = data_df.loc[i-1, 'pe_oi_change']
    if data_df.loc[i, 'ce_oi_change'] == 1 and data_df.loc[i-1, 'ce_oi_change'] not in [0, 1]:
        data_df.loc[i, 'ce_oi_change'] = data_df.loc[i-1, 'ce_oi_change']

In [7]:
data_df.to_csv('traning_data1.csv', index=False)

In [8]:
columns_order = [
            'unique_key', 'date', 'time', 'direction', 'ce_pe_oi_ratio', 'prev_ce_pe_oi_ratio',
            'ce_beta', 'ce_oi_change', 'pre_ce_oi_change', 'first_ce_oi_change',
            'pe_beta', 'pe_oi_change', 'pre_pe_oi_change', 'first_pe_oi_change', 'state', 'action'
        ]
        
df = data_df[columns_order]
df.shape

(3684, 16)

In [9]:
subset = ['time', 'direction', 'ce_pe_oi_ratio', 'prev_ce_pe_oi_ratio',
            'ce_beta', 'ce_oi_change', 'pre_ce_oi_change', 'first_ce_oi_change',
            'pe_beta', 'pe_oi_change', 'pre_pe_oi_change', 'first_pe_oi_change', 'action']
df = df.drop_duplicates(subset=subset)
df

,unique_key,date,time,direction,ce_pe_oi_ratio,prev_ce_pe_oi_ratio,ce_beta,ce_oi_change,pre_ce_oi_change,first_ce_oi_change,pe_beta,pe_oi_change,pre_pe_oi_change,first_pe_oi_change,state,action
0,2.025040e+11,04/04/25 9:15,915,Down,0.4909,0.0000,0.23,1.0000,0.0000,1.0000,-9.58,1.0000,0.0000,1.0000,No Trade,No Trade
15,2.025040e+11,04/04/25 9:15,915,Down,0.8516,0.0000,0.23,1.6472,0.0000,1.6472,-9.58,0.9495,0.0000,0.9495,No Trade,Sell
16,2.025040e+11,04/04/25 9:15,915,Down,0.8516,0.0000,0.23,1.6472,0.0000,1.6472,-9.58,0.9495,0.0000,0.9495,Sell,Sell Hold
46,2.025040e+11,04/04/25 9:15,915,Down,1.1342,0.0000,0.23,2.1089,0.0000,2.1089,-9.58,0.9127,0.0000,0.9127,Sell,Sell Hold
51,2.025040e+11,04/04/25 9:20,920,Down,1.1342,0.4909,32.58,2.1089,2.1089,2.1089,-44.70,0.9127,0.9127,0.9127,Sell,Sell Hold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598,2.025040e+11,04/04/25 15:15,1515,Up,3.8318,3.8893,2.34,0.9840,0.9502,3.7344,-3.72,0.9644,0.9644,0.4784,No Trade,No Trade
3599,2.025040e+11,04/04/25 15:15,1515,Up,3.8810,3.8893,2.34,0.9840,0.9502,3.7344,-3.72,0.9873,0.9522,0.4724,No Trade,No Trade
3624,2.025040e+11,04/04/25 15:20,1520,Up,3.8810,3.8940,5.50,0.9840,0.9840,3.7344,-1.72,0.9873,0.9873,0.4724,No Trade,No Trade
3627,2.025040e+11,04/04/25 15:20,1520,Up,3.8701,3.8940,5.50,0.9786,0.9630,3.6545,-1.72,0.9814,0.9689,0.4636,No Trade,No Trade


In [10]:
df.to_csv('traning_data2.csv', index=False)

In [12]:
ai_data_generator.load_traning_data('traning_data2.csv')

2025-04-07 00:13:29,296 - root - DEBUG - ✅ Connected to PostgreSQL successfully!
2025-04-07 00:13:29,313 - root - DEBUG - ✅ Query executed successfully.
2025-04-07 00:13:29,315 - root - DEBUG - 🔌 Database connection closed.
2025-04-07 00:13:29,315 - root - ERROR - Saved traning data


In [25]:
traning_df = ai_data_generator.fetch_traning_data(current_time)

2025-04-05 18:41:09,947 - root - DEBUG - ✅ Connected to PostgreSQL successfully!
2025-04-05 18:41:09,962 - root - DEBUG - 🔌 Database connection closed.


In [26]:
traning_df

,unique_key,date,time,direction,ce_pe_oi_ratio,prev_ce_pe_oi_ratio,ce_beta,ce_oi_change,pre_ce_oi_change,first_ce_oi_change,pe_beta,pe_oi_change,pre_pe_oi_change,first_pe_oi_change,action
0,202504040915,2025-04-04 09:15:00,915,1,0.49,0.00,0.23,1.00,0.00,1.00,-9.58,1.00,0.00,1.00,0
1,202504040915,2025-04-04 09:15:00,915,1,0.85,0.00,0.23,1.65,0.00,1.65,-9.58,0.95,0.00,0.95,2
2,202504040915,2025-04-04 09:15:00,915,1,1.13,0.00,0.23,2.11,0.00,2.11,-9.58,0.91,0.00,0.91,2
3,202504040920,2025-04-04 09:20:00,920,1,1.13,0.49,32.58,2.11,2.11,2.11,-44.70,0.91,0.91,0.91,2
4,202504040920,2025-04-04 09:20:00,920,1,1.27,0.49,32.58,1.12,2.36,2.36,-44.70,0.91,0.91,0.91,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,202504041515,2025-04-04 15:15:00,1515,1,3.83,3.89,2.34,0.98,0.95,3.73,-3.72,0.96,0.96,0.48,0
212,202504041515,2025-04-04 15:15:00,1515,1,3.88,3.89,2.34,0.98,0.95,3.73,-3.72,0.99,0.95,0.47,0
213,202504041520,2025-04-04 15:20:00,1520,1,3.88,3.89,5.50,0.98,0.98,3.73,-1.72,0.99,0.99,0.47,0
214,202504041520,2025-04-04 15:20:00,1520,1,3.87,3.89,5.50,0.98,0.96,3.65,-1.72,0.98,0.97,0.46,0


In [11]:
traning_df.columns

Index(['unique_key', 'date', 'time', 'direction', 'ce_pe_oi_ratio',
       'prev_ce_pe_oi_ratio', 'ce_beta', 'ce_oi_change', 'pre_ce_oi_change',
       'first_ce_oi_change', 'pe_beta', 'pe_oi_change', 'pre_pe_oi_change',
       'first_pe_oi_change', 'action'],
      dtype='object')

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn
print(sklearn.__version__)

1.6.1


In [13]:
features = ['time', 'direction', 'ce_pe_oi_ratio', 'prev_ce_pe_oi_ratio', 
            'ce_beta', 'ce_oi_change', 'pre_ce_oi_change', 'first_ce_oi_change', 
            'pe_beta', 'pe_oi_change', 'pre_pe_oi_change', 'first_pe_oi_change']

target = 'action'  # Buy/Sell/Hold

X_train, X_test, y_train, y_test = train_test_split(traning_df[features], traning_df[target], test_size=0.2, random_state=42)

print(y_train.unique())
# Train XGBoost model
model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Performance report
print(classification_report(y_test, y_pred))

[1 0 2]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       386
           1       1.00      1.00      1.00        67
           2       1.00      1.00      1.00       284

    accuracy                           1.00       737
   macro avg       1.00      1.00      1.00       737
weighted avg       1.00      1.00      1.00       737



In [14]:
X_test

,time,direction,ce_pe_oi_ratio,prev_ce_pe_oi_ratio,ce_beta,ce_oi_change,pre_ce_oi_change,first_ce_oi_change,pe_beta,pe_oi_change,pre_pe_oi_change,first_pe_oi_change
2023,1235,1,3.23,3.17,3.54,1.01,1.02,3.99,-13.08,1.01,1.00,0.61
2587,1330,1,3.47,3.44,19.99,1.00,1.01,4.23,-12.10,1.00,1.00,0.60
3222,1435,1,4.06,3.99,8.60,1.00,0.99,4.40,-6.69,0.99,0.98,0.53
1263,1115,1,2.58,2.53,39.90,1.00,1.00,3.36,-28.06,1.00,0.99,0.64
781,1030,1,3.00,2.93,28.17,1.00,1.02,3.90,-23.91,1.00,0.99,0.64
...,...,...,...,...,...,...,...,...,...,...,...,...
2423,1310,1,3.45,3.39,13.97,1.02,1.02,4.19,-5.29,0.99,1.00,0.60
1717,1205,1,3.04,3.00,41.56,1.00,1.01,3.83,-22.94,1.00,0.99,0.62
25,915,1,0.85,0.00,0.23,1.65,0.00,1.65,-9.58,0.95,0.00,0.95
2228,1255,1,3.41,3.37,3.10,1.00,1.02,4.16,-5.21,1.00,1.01,0.60


In [17]:
y_pred

array([0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1, 1, 2, 0, 0,
       0, 2, 1, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0,
       0, 2, 0, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0,
       0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 1, 1, 0, 2, 2, 2, 0, 0, 0,
       0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0,
       2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0,
       2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 2, 1, 2, 0, 0, 2, 2,
       2, 1, 2, 1, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 0, 0,
       2, 0, 2, 0, 0, 0, 2, 1, 2, 1, 2, 2, 0, 0, 0, 2, 1, 0, 0, 0, 2, 2,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0,
       2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0,